In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json

# read and merge 3 dfs

In [2]:
df1 = pd.read_csv('data/chall.csv',low_memory=False)
print(df1.shape)

(10446, 1552)


In [3]:
df2 = pd.read_csv('data/gm.csv',low_memory=False)
print(df2.shape)

(10724, 1552)


In [4]:
df3 = pd.read_csv('data/master.csv',low_memory=False)
print(df3.shape)

(12617, 1564)


In [5]:
df = pd.concat([df1,df2,df3])
print(df.shape)

(33787, 1564)


# Drop columns of single distinct value or >25% null

In [6]:
# keep only classic games

df = df[df['gameMode'] == "CLASSIC"]

In [7]:
# drop if one distinct value

for col in df.columns:
    if len(df[col].unique())==1:
        df = df.drop(col,axis=1)

In [8]:
# drop if null values are more than 25% of column

for col in df.columns:
    if (np.mean(df[col]!=df[col]) > .25):
        df = df.drop(col,axis=1)

In [9]:
print(df.shape)

(29473, 1186)


In [10]:
for col in df.columns:
    print(col + ": " + str(df[col].dtype))

gameId: float64
gameCreation: float64
gameDuration: float64
queueId: float64
gameVersion: object
teams_0_win: object
teams_0_firstBlood: bool
teams_0_firstTower: bool
teams_0_firstInhibitor: bool
teams_0_firstBaron: bool
teams_0_firstDragon: bool
teams_0_firstRiftHerald: bool
teams_0_towerKills: float64
teams_0_inhibitorKills: float64
teams_0_baronKills: float64
teams_0_dragonKills: float64
teams_0_riftHeraldKills: float64
teams_1_win: object
teams_1_firstBlood: bool
teams_1_firstTower: bool
teams_1_firstInhibitor: bool
teams_1_firstBaron: bool
teams_1_firstDragon: bool
teams_1_firstRiftHerald: bool
teams_1_towerKills: float64
teams_1_inhibitorKills: float64
teams_1_baronKills: float64
teams_1_dragonKills: float64
teams_1_riftHeraldKills: float64
participants_0_championId: float64
participants_0_spell1Id: float64
participants_0_spell2Id: float64
participants_0_stats_win: bool
participants_0_stats_item0: float64
participants_0_stats_item1: float64
participants_0_stats_item2: float64
par

In [11]:
# fill after to allow int64 reconversion
df = df.fillna(0)

In [12]:
# drop if null values are more than 25% of column

for col in df.columns:
    if (df[col].dtypes == np.float64) and (sum(df[col]%1) == 0):
        df[col] = df[col].astype(np.int64)
    if (df[col].dtypes == bool):
        df[col] = df[col].astype(int)

In [13]:
df[['teams_0_win','teams_1_win']] = df[['teams_0_win','teams_1_win']].replace(['Win','Fail'],[1,0])

In [14]:
df

,gameId,gameCreation,gameDuration,queueId,gameVersion,teams_0_win,teams_0_firstBlood,teams_0_firstTower,teams_0_firstInhibitor,teams_0_firstBaron,...,participants_7_stats_wardsPlaced,participants_7_stats_wardsKilled,participants_8_stats_neutralMinionsKilledTeamJungle,participants_8_stats_neutralMinionsKilledEnemyJungle,participants_8_stats_wardsPlaced,participants_8_stats_wardsKilled,participants_9_stats_neutralMinionsKilledTeamJungle,participants_9_stats_neutralMinionsKilledEnemyJungle,participants_9_stats_wardsPlaced,participants_9_stats_wardsKilled
2,3425927177,1589811429884,1548,420,10.10.320.3039,1,1,1,1,1,...,8,2,92,0,3,3,1,0,9,0
3,3463315468,1592285561264,1499,420,10.12.324.5925,0,0,0,0,0,...,21,6,93,16,2,5,0,4,3,5
4,3490775054,1594487307024,2300,420,10.14.327.8505,0,0,1,0,1,...,6,5,102,3,6,18,29,1,13,8
5,3449782290,1591330333433,1413,420,10.11.322.2991,0,1,0,0,0,...,10,2,0,0,26,4,0,0,9,7
6,3482451986,1593821192664,1608,420,10.13.326.4870,1,1,0,1,1,...,8,2,4,0,8,0,0,0,20,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12612,3487563744,1594224435270,1549,420,10.14.327.1684,1,1,0,1,1,...,25,7,4,0,6,4,4,0,8,2
12613,3480158177,1593634490053,762,420,10.13.326.4870,1,0,1,1,0,...,4,1,0,0,2,0,0,0,1,0
12614,3326738405,1584385654501,1407,420,10.5.312.392,1,1,1,1,1,...,7,0,0,0,6,2,0,0,8,1
12615,3479175144,1593561996811,2128,420,10.13.326.4870,1,0,0,1,0,...,12,16,102,10,6,2,0,0,29,6


In [15]:
#df.to_csv('data/clean.csv', index=False)